# Prompt Injection Attacker — single notebook
**Author:** Nebiyu Zewge
**Date:** 2025-11-04

**WARNING:** This notebook will generate malicious prompts and may produce harmful outputs. Use only for research in a safe environment. Do not run on public endpoints or use generated prompts for malicious activity.
